In [1]:
from datasets import load_dataset

ds = load_dataset("katanaml-org/invoices-donut-data-v1")

/home/lucifer/Documents/Invoice_Generation/myvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ds['train'][0]['ground_truth']

'{"gt_parse": {"header": {"invoice_no": "40378170", "invoice_date": "10/15/2012", "seller": "Patel, Thompson and Montgomery 356 Kyle Vista New James, MA 46228", "client": "Jackson, Odonnell and Jackson 267 John Track Suite 841 Jenniferville, PA 98601", "seller_tax_id": "958-74-3511", "client_tax_id": "998-87-7723", "iban": "GB77WRBQ31965128414006"}, "items": [{"item_desc": "Leed\'s Wine Companion Bottle Corkscrew Opener Gift Box Set with Foil Cutter", "item_qty": "1,00", "item_net_price": "7,50", "item_net_worth": "7,50", "item_vat": "10%", "item_gross_worth": "8,25"}], "summary": {"total_net_worth": "$7,50", "total_vat": "$0,75", "total_gross_worth": "$8,25"}}}'

In [2]:
img2 = ds['test'][5]['image']
img2.save("test_image_5.png")

In [3]:
img2 = ds['test'][8]['image']
img2.save("test_image_8.png")

In [2]:
import sys, site, platform
print("Executable:", sys.executable)
print("Python version:", sys.version)
print("Platform:", platform.platform())
import pkgutil
print("Pillow present in working interpreter?:", pkgutil.find_loader("PIL") is not None)


Executable: /home/lucifer/Documents/Invoice_Generation/myvenv/bin/python
Python version: 3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]
Platform: Linux-6.14.0-24-generic-x86_64-with-glibc2.39
Pillow present in working interpreter?: True


/tmp/ipykernel_4724/613248888.py:6: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  print("Pillow present in working interpreter?:", pkgutil.find_loader("PIL") is not None)


In [3]:
# ds['train'][0]['img']

In [4]:
schema_dict = {
    "header": {
        "invoice_no": "Invoice number of the document",
        "invoice_date": "Date when the invoice was issued",
        "seller": "Full address and name of the seller",
        "client": "Full address and name of the client",
        "seller_tax_id": "Tax identification number of the seller",
        "client_tax_id": "Tax identification number of the client",
        "iban": "Bank IBAN number of the seller"
    },
    "items": [
        {
            "item_desc": "Description of the service or product sold",
            "item_qty": "Quantity of the item (usually in units or pieces)",
            "item_net_price": "Unit price excluding VAT",
            "item_net_worth": "Total price excluding VAT",
            "item_vat": "VAT rate applied to this item",
            "item_gross_worth": "Total price including VAT"
        }
    ],
    "summary": {
        "total_net_worth": "Total net amount before VAT",
        "total_vat": "Total VAT amount",
        "total_gross_worth": "Final total amount including VAT"
    }
}

In [5]:
import json
instruction = f"""You are a specialized in invoice and your role is to extract information from any invoice that is provided to you in the following valid json format. if the corresponding value is not present, leave the key with empty string.

{json.dumps(schema_dict)}

Fill the keys only when the information is available.
"""

In [6]:
instruction

'You are a specialized in invoice and your role is to extract information from any invoice that is provided to you in the following valid json format. if the corresponding value is not present, leave the key with empty string.\n\n{"header": {"invoice_no": "Invoice number of the document", "invoice_date": "Date when the invoice was issued", "seller": "Full address and name of the seller", "client": "Full address and name of the client", "seller_tax_id": "Tax identification number of the seller", "client_tax_id": "Tax identification number of the client", "iban": "Bank IBAN number of the seller"}, "items": [{"item_desc": "Description of the service or product sold", "item_qty": "Quantity of the item (usually in units or pieces)", "item_net_price": "Unit price excluding VAT", "item_net_worth": "Total price excluding VAT", "item_vat": "VAT rate applied to this item", "item_gross_worth": "Total price including VAT"}], "summary": {"total_net_worth": "Total net amount before VAT", "total_vat"

In [7]:
from dotenv import load_dotenv
load_dotenv('/home/lucifer/Documents/Invoice_Generation/.env')

True

In [8]:
import os 

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

In [9]:
from huggingface_hub import login 

login(token=HUGGINGFACE_TOKEN)

In [10]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from transformers import BitsAndBytesConfig

In [13]:
!nvidia-smi

Mon Jul 21 19:30:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   55C    P8              7W /   60W |      18MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [14]:
model_id = "Adarsh203/qwen-2.5-vl-3b-invoices"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto")
processor = AutoProcessor.from_pretrained(model_id,use_fast=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.07s/it]


In [15]:
!nvidia-smi

Mon Jul 21 19:35:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   56C    P8              8W /   60W |    2635MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [17]:
import gradio as gr
from PIL import Image
import os
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import json

# Default prompt
instruction = f"""You are a specialized in invoice and your role is to extract information from any invoice that is provided to you in the following valid json format. if the corresponding value is not present, leave the key with empty string.

{json.dumps(schema_dict)}

Fill the keys only when the information is available.
"""

# Function to create messages
def create_messages(image_path, prompt):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path,
                    "resized_height": 640,
                    "resized_width": 640
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]
    return messages

# Inference logic
def infer(messages):
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)

    generated_ids = model.generate(**inputs, max_new_tokens=2048)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text[0]

# Gradio handler
def invoice_parser(image: Image.Image):
    image_path = os.path.join("inference", "test_image.png")
    os.makedirs("inference", exist_ok=True)
    image.save(image_path)

    messages = create_messages(image_path=image_path, prompt=instruction)
    output = infer(messages)
    return output

def reset_fields():
    return None, None

# Gradio UI
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Upload Invoice")
            image_input = gr.Image(type="pil", label="Invoice Image")
            submit_btn = gr.Button("Submit")
            reset_btn = gr.Button("Reset")

        with gr.Column(scale=1):
            gr.Markdown("### Parsed Invoice Details")
            output_text = gr.Textbox(label="Extracted Data", lines=10)

    submit_btn.click(fn=invoice_parser, inputs=image_input, outputs=output_text)
    reset_btn.click(fn=reset_fields, inputs=[], outputs=[image_input, output_text])

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://63b8de3f35e1491651.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
import torch

In [18]:
!nvidia-smi

Mon Jul 21 19:38:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   59C    P8              8W /   60W |    2783MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
